# MovieLens Dataset: Exploration and Cleaning

In [1]:
import pandas as pd
import warnings
import matplotlib.pyplot as plt
%matplotlib inline
warnings.filterwarnings('ignore')

## Download Instructions

1. Go to the [Kaggle Movies Dataset](https://www.kaggle.com/rounakbanik/the-movies-dataset)
2. Download `movies_metadata.csv` and place it in the same folder

In [2]:
# import csv file (note: ~34MB)
movies_file = 'movies_metadata.csv'
movies_rawdata = pd.read_csv(movies_file)

In [3]:
len(movies_rawdata) # 45k rows is too much. We'll have to cut this down.

45466

In [4]:
movies_rawdata.columns.values # column names

array(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage',
       'id', 'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'], dtype=object)

In [5]:
movies_rawdata.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


## Cleaning: Subset Data

In [6]:
movies = movies_rawdata.copy()

# convert budget column to int type
movies[['budget']] = movies[['budget']].apply(pd.to_numeric, errors='coerce').astype(str).astype(float)
movies = movies[movies.budget.notna()]
movies[['budget']] = movies[['budget']].astype(int)

# only filter out the big/blockbuster movies available in English
movies = movies[movies.budget > 100000][movies.revenue > 100000][movies.runtime > 0]
movies = movies[movies.spoken_languages.str.contains("English")]

# drop extraneous columns
keep_columns = ['title','vote_average','budget','revenue','runtime','genres','release_date','production_companies','vote_count','popularity','original_language'] 
movies = movies[keep_columns]

## Cleaning: Create new columns (one-hot encoding)

In [7]:
# define interesting genres & production cos
genres = ['Action', 'Animation', 'Comedy', 'Crime', 'Drama', 'Family', 'Fantasy', 'History', 'Mystery', 'Romance', 'Science Fiction', 'Thriller', 'War']
cos = ['Pixar Animation Studios', 'Warner Bros.', 'Paramount Pictures', 'Columbia Pictures', 'Metro-Goldwyn-Mayer', 'Universal Pictures', 'Walt Disney Pictures', 'Marvel Studios', 'DC Comics']

In [8]:
# helper function for extracting genres & prod companies from json
def deconstruct_json(json_str):
    item_list = []
    for item in eval(json_str):
        item_list.append(item['name'])
    
    return ','.join(item_list)

In [9]:
# create column for release_year based on release_date, and only keep movies produced after 1951
movies['release_year'] = movies['release_date'].str.slice(0,4).astype(int)
movies = movies[movies.release_year > 1951]

movies['genres'] = movies['genres'].apply(deconstruct_json)
movies['production_companies'] = movies['production_companies'].apply(deconstruct_json)
for genre in genres:
    movies[genre.lower().replace("science fiction", "scifi")] = movies['genres'].str.contains(genre).astype(int)
for co in cos:
    movies[co.lower().replace(" ", "").replace(".", "").replace("-", "")] = movies['production_companies'].str.contains(co).astype(int)

## Visualization
### Blockbuster Movies in 2017

In [10]:
movies[movies.release_year == max(movies['release_year'])].sort_values(by='revenue', ascending=False)

,title,vote_average,budget,revenue,runtime,genres,release_date,production_companies,vote_count,popularity,...,war,pixaranimationstudios,warnerbros,paramountpictures,columbiapictures,metrogoldwynmayer,universalpictures,waltdisneypictures,marvelstudios,dccomics
42222,Beauty and the Beast,6.8,160000000,1.262886e+09,129.0,"Family,Fantasy,Romance",2017-03-16,"Walt Disney Pictures,Mandeville Films",5530.0,287.253654,...,0,0,0,0,0,0,0,1,0,0
43255,The Fate of the Furious,6.8,250000000,1.238765e+09,136.0,"Action,Crime,Thriller",2017-04-12,"Universal Pictures,Original Film,One Race Films",3803.0,48.573287,...,0,0,0,0,0,0,1,0,0,0
44009,Despicable Me 3,6.2,80000000,1.020063e+09,96.0,"Action,Animation,Adventure,Family,Comedy",2017-06-15,Illumination Entertainment,2002.0,36.631519,...,0,0,0,0,0,0,0,0,0,0
26566,Guardians of the Galaxy Vol. 2,7.6,200000000,8.634161e+08,137.0,"Action,Adventure,Comedy,Science Fiction",2017-04-19,"Walt Disney Pictures,Marvel Studios",4858.0,185.331,...,0,0,0,0,0,0,0,1,1,0
33356,Wonder Woman,7.2,149000000,8.205804e+08,141.0,"Action,Adventure,Fantasy",2017-05-30,"Dune Entertainment,Atlas Entertainment,Warner ...",5025.0,294.337037,...,0,0,1,0,0,0,0,0,0,0
26560,Pirates of the Caribbean: Dead Men Tell No Tales,6.6,230000000,7.941920e+08,129.0,"Adventure,Action,Fantasy,Comedy",2017-05-23,"Walt Disney Pictures,Jerry Bruckheimer Films,I...",2814.0,133.828,...,0,0,0,0,0,0,0,1,0,0
42170,Logan,7.6,97000000,6.168018e+08,137.0,"Action,Drama,Science Fiction",2017-02-28,"Twentieth Century Fox Film Corporation,Donners...",6310.0,54.581997,...,0,0,0,0,0,0,0,0,0,0
44842,Transformers: The Last Knight,6.2,260000000,6.049421e+08,149.0,"Action,Science Fiction,Thriller,Adventure",2017-06-21,"Paramount Pictures,Di Bonaventura Pictures,Ang...",1440.0,39.186819,...,0,0,0,1,0,0,0,0,0,0
42171,Kong: Skull Island,6.2,185000000,5.666528e+08,118.0,"Action,Adventure,Fantasy",2017-03-08,"Legendary Pictures,Warner Bros.",3161.0,29.429112,...,0,0,1,0,0,0,0,0,0,0
44678,Dunkirk,7.5,100000000,5.198769e+08,107.0,"Action,Drama,History,Thriller,War",2017-07-19,"Canal+,Studio Canal,Warner Bros.,Syncopy,RatPa...",2712.0,30.938854,...,1,0,1,0,0,0,0,0,0,0


### Number of movies produced by each major production co

In [11]:
for co in cos:
    print(co, sum(movies[co.lower().replace(" ", "").replace(".", "").replace("-", "")]))

Pixar Animation Studios 18
Warner Bros. 414
Paramount Pictures 329
Columbia Pictures 316
Metro-Goldwyn-Mayer 130
Universal Pictures 383
Walt Disney Pictures 129
Marvel Studios 15
DC Comics 16


### Interesting Movie Collections

#### Movies by Marvel Studios

In [12]:
movies[movies.production_companies.str.contains('Marvel Studios')]

,title,vote_average,budget,revenue,runtime,genres,release_date,production_companies,vote_count,popularity,...,war,pixaranimationstudios,warnerbros,paramountpictures,columbiapictures,metrogoldwynmayer,universalpictures,waltdisneypictures,marvelstudios,dccomics
12588,Iron Man,7.4,140000000,5.851742e+08,126.0,"Action,Science Fiction,Adventure",2008-04-30,"Paramount Pictures,Marvel Studios",8951.0,22.0731,...,0,0,0,1,0,0,0,0,1,0
12700,The Incredible Hulk,6.1,150000000,1.637121e+08,114.0,"Science Fiction,Action,Adventure",2008-06-12,"Universal Pictures,Marvel Studios,Valhalla Mot...",3086.0,19.1255,...,0,0,0,0,0,0,1,0,1,0
15153,Iron Man 2,6.6,200000000,6.239333e+08,124.0,"Adventure,Action,Science Fiction",2010-04-28,Marvel Studios,6969.0,19.0833,...,0,0,0,0,0,0,0,0,1,0
16974,Thor,6.6,150000000,4.493266e+08,115.0,"Adventure,Fantasy,Action",2011-04-21,Marvel Studios,6678.0,29.1585,...,0,0,0,0,0,0,0,0,1,0
17444,Captain America: The First Avenger,6.6,140000000,3.705698e+08,124.0,"Action,Adventure,Science Fiction",2011-07-22,"Paramount Pictures,Marvel Studios",7174.0,19.3236,...,0,0,0,1,0,0,0,0,1,0
17818,The Avengers,7.4,220000000,1.519558e+09,143.0,"Science Fiction,Action,Adventure",2012-04-25,"Paramount Pictures,Marvel Studios",12000.0,89.8876,...,0,0,0,1,0,0,0,0,1,0
20830,Iron Man 3,6.8,200000000,1.215440e+09,130.0,"Action,Adventure,Science Fiction",2013-04-18,Marvel Studios,8951.0,23.7212,...,0,0,0,0,0,0,0,0,1,0
21941,Thor: The Dark World,6.8,170000000,6.445714e+08,112.0,"Action,Adventure,Fantasy",2013-10-29,Marvel Studios,4873.0,34.9054,...,0,0,0,0,0,0,0,0,1,0
23053,Captain America: The Winter Soldier,7.6,170000000,7.147666e+08,136.0,"Action,Adventure,Science Fiction",2014-03-20,Marvel Studios,5881.0,18.7177,...,0,0,0,0,0,0,0,0,1,0
23753,Guardians of the Galaxy,7.9,170000000,7.733286e+08,121.0,"Action,Science Fiction,Adventure",2014-07-30,"Marvel Studios,Moving Picture Company (MPC),Bu...",10014.0,53.2916,...,0,0,0,0,0,0,0,0,1,0


#### Titanic movies (both versions)

In [13]:
movies[movies.title.str.contains('Titanic')]

,title,vote_average,budget,revenue,runtime,genres,release_date,production_companies,vote_count,popularity,...,war,pixaranimationstudios,warnerbros,paramountpictures,columbiapictures,metrogoldwynmayer,universalpictures,waltdisneypictures,marvelstudios,dccomics
1639,Titanic,7.5,200000000,1.845034e+09,194.0,"Drama,Romance,Thriller",1997-11-18,"Paramount Pictures,Twentieth Century Fox Film ...",7770.0,26.8891,...,0,0,0,1,0,0,0,0,0,0
3285,Titanic,6.2,1805000,4.905000e+06,98.0,"Drama,Action,Romance",1953-04-11,Twentieth Century Fox Film Corporation,27.0,12.9071,...,0,0,0,0,0,0,0,0,0,0


#### The James Bond Collection

In [14]:
m = movies_rawdata[movies_rawdata.overview.notna()]
m[m.overview.str.contains('james bond', case=False)].sort_values(by='release_date', ascending=False).head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
44318,False,NaN,0,"[{'id': 99, 'name': 'Documentary'}, {'id': 35,...",NaN,438493,tt6110504,en,Becoming Bond,The stranger-than-fiction true story of George...,...,2017-03-11,0.0,91.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,"The name’s Lazenby, George Lazenby",Becoming Bond,False,7.1,10.0
18006,False,"{'id': 86117, 'name': 'Johnny English Collecti...",45000000,"[{'id': 80, 'name': 'Crime'}, {'id': 12, 'name...",http://www.johnnyenglishreborn.com/,58233,tt1634122,en,Johnny English Reborn,The most prominent heads of state in the world...,...,2011-09-15,160078586.0,101.0,"[{'iso_639_1': 'de', 'name': 'Deutsch'}, {'iso...",Released,One man. One mission. No chance.,Johnny English Reborn,False,6.0,1026.0
13061,False,"{'id': 645, 'name': 'James Bond Collection', '...",200000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",http://www.mgm.com/view/movie/234/Quantum-of-S...,10764,tt0830515,en,Quantum of Solace,Quantum of Solace continues the adventures of ...,...,2008-10-30,586090727.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,"For love, for hate, for justice, for revenge.",Quantum of Solace,False,6.1,3015.0
23135,False,NaN,0,"[{'id': 99, 'name': 'Documentary'}]",NaN,34079,tt0353252,en,Bond Girls Are Forever,Through vintage film clips of past Bond movie ...,...,2002-11-06,0.0,46.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Bond Girls Are Forever,False,5.2,5.0
14473,False,"{'id': 2248, 'name': 'Torrente Collection', 'p...",0,"[{'id': 35, 'name': 'Comedy'}, {'id': 80, 'nam...",NaN,1284,tt0257231,es,Torrente 2: Misión en Marbella,For this second film in the cult comedy series...,...,2001-03-30,0.0,99.0,"[{'iso_639_1': 'es', 'name': 'Español'}]",Released,NaN,Torrente 2: Mission in Marbella,False,6.3,41.0


### International Movies: Blockbusters originating from non-English versions (2011-2017)

In [15]:
movies[movies.release_year > 2010][~movies.original_language.str.contains('en')].sort_values(by='release_date', ascending=False)

,title,vote_average,budget,revenue,runtime,genres,release_date,production_companies,vote_count,popularity,...,war,pixaranimationstudios,warnerbros,paramountpictures,columbiapictures,metrogoldwynmayer,universalpictures,waltdisneypictures,marvelstudios,dccomics
42018,Kung Fu Yoga,5.9,65000000,254212245.0,130.0,"Mystery,Action,Adventure,Comedy",2017-01-27,"Shinework Media,Taihe Entertainment",50.0,17.630487,...,0,0,0,0,0,0,0,0,0,0
40675,Railroad Tigers,6.2,50000000,102205175.0,124.0,"Action,Comedy,War",2016-12-23,"Shanghai Film Group,Sparkle Roll Media,Beijing...",22.0,3.798504,...,1,0,0,0,0,0,0,0,0,0
41516,Aanandam,6.4,450000,2300000.0,134.0,"Adventure,Comedy,Romance",2016-10-21,"Cast N' Crew,Habit of Life",7.0,0.354748,...,0,0,0,0,0,0,0,0,0,0
42808,Personal Shopper,5.8,6000000,1305195.0,105.0,"Drama,Mystery,Thriller",2016-10-19,"ARTE France Cinéma,Sirena Film,Scope Pictures,...",239.0,11.186981,...,0,0,0,0,0,0,0,0,0,0
41258,Smoke & Mirrors,6.7,5000000,2387127.0,123.0,"History,Thriller",2016-09-23,"Atípica Films,Atresmedia Cine,Sacromonte Films...",50.0,3.138159,...,0,0,0,0,0,0,0,0,0,0
40224,Shin Godzilla,6.6,15000000,77000000.0,120.0,"Action,Adventure,Drama,Horror,Science Fiction",2016-07-29,"Cine Bazar,Toho Pictures",152.0,9.285519,...,0,0,0,0,0,0,0,0,0,0
38801,Toni Erdmann,7.3,3537415,1390172.0,162.0,"Comedy,Drama",2016-07-14,"Sony Pictures Classics,Arte,Komplizen Film,Wes...",270.0,9.117054,...,0,0,0,0,0,0,0,0,0,0
39495,Sultan,7.0,10400000,102000000.0,170.0,"Drama,Action",2016-07-06,Yash Raj Films,71.0,6.070606,...,0,0,0,0,0,0,0,0,0,0
43738,In the Forests of Siberia,7.3,3500000,1492523.0,105.0,"Drama,Adventure",2016-06-15,Nord-Ouest Films,34.0,1.333969,...,0,0,0,0,0,0,0,0,0,0
39718,Robinson Crusoe,5.8,13500000,3342697.0,90.0,"Adventure,Animation,Comedy,Family",2016-02-04,"StudioCanal,uFilm,nWave Pictures,Illuminata Pi...",85.0,8.969705,...,0,0,0,0,0,0,0,0,0,0


#### Hey, look! We even have the 3 idiots -

In [16]:
movies[movies.title.str.contains('idiots', case=False)]

,title,vote_average,budget,revenue,runtime,genres,release_date,production_companies,vote_count,popularity,...,war,pixaranimationstudios,warnerbros,paramountpictures,columbiapictures,metrogoldwynmayer,universalpictures,waltdisneypictures,marvelstudios,dccomics
14732,3 Idiots,7.8,9000000,70000000.0,170.0,"Drama,Comedy,Romance",2009-12-23,Vidhu Vinod Chopra Productions,850.0,7.67562,...,0,0,0,0,0,0,0,0,0,0


## Save cleaned dataframe to csv

In [17]:
movies = movies.sort_values(by=['release_year','revenue'], ascending=[False, False])
movies.to_csv('movies.csv', index=False)